Download data from DMI:
=======================
https://dmigw.govcloud.dk/v2/metObs/bulk/?api-key=d40321ee-7de5-4417-a0bf-108bd34061ab

https://dmigw.govcloud.dk/v2/climateData/bulk/?api-key=fa9056ec-2f41-4042-828b-91750e966966

In [2]:
forecast_api_key = 'e55dced6-70fb-44f1-a658-cdef62c74b6a'
climate_api_key = 'fa9056ec-2f41-4042-828b-91750e966966'
metobs_api_key = 'd40321ee-7de5-4417-a0bf-108bd34061ab'

In [12]:
import os
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import plotly.express as px
import plotly.graph_objects as go

In [4]:
def load_json_files(folder_path):
    data = []  # List to hold all records

    # Iterate over all .txt files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # Process only .txt files
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                for line in file:
                    if line.strip():  # Check if the line is not empty
                        try:
                            # Parse each line as a separate JSON object
                            json_obj = json.loads(line)
                            
                            # Extract specific fields from the JSON
                            properties = json_obj.get('properties', {})
                            geometry = json_obj.get('geometry', {})
                            
                            # Unpack coordinates (assuming it's a Polygon)
                            coordinates = geometry.get('coordinates', [[]])
                            flat_coords = coordinates[0] if coordinates else []

                            # Append unpacked data
                            data.append({
                                'file': filename,
                                'calculatedAt': properties.get('calculatedAt'),
                                'cellId': properties.get('cellId'),
                                'created': properties.get('created'),
                                'from': properties.get('from'),
                                'parameterId': properties.get('parameterId'),
                                'qcStatus': properties.get('qcStatus'),
                                'timeResolution': properties.get('timeResolution'),
                                'to': properties.get('to'),
                                'value': properties.get('value'),
                                'geometry_type': geometry.get('type'),
                                'coordinates': flat_coords,  # Store unpacked coordinates
                                'feature_id': json_obj.get('id')  # Extract feature ID
                            })
                        except json.JSONDecodeError as e:
                            print(f"Error decoding JSON in file {filename}: {e}")

    # Convert the list of dictionaries into a dataframe
    df = pd.DataFrame(data)


    return df

In [5]:
# Path to the folder containing the .txt files
folder_path = '/Users/johan/Downloads/all'

# Load the JSON files and create the dataframe
df = load_json_files(folder_path)

# Display the dataframe
display(df.head())  # Print first few rows

# Save to a CSV if needed
# df.to_csv('unpacked_data.csv', index=False)

,file,calculatedAt,cellId,created,from,parameterId,qcStatus,timeResolution,to,value,geometry_type,coordinates,feature_id
0,2024-01-19.txt,2024-01-21T15:07:07.298000,20km_606_66,2024-01-22T12:23:25.352484+00:00,2024-01-19T23:00:00+00:00,temp_soil_30,none,hour,2024-01-20T00:00:00+00:00,2.4,Polygon,"[[11.4807, 54.6619], [11.7904, 54.6552], [11.8...",0020e993-841f-dfe0-17aa-fc258d6b230b
1,2024-01-19.txt,2024-02-27T13:36:45.153000,20km_618_70,2024-02-27T13:39:41.416613+00:00,2024-01-19T23:00:00+00:00,min_temp,manual,hour,2024-01-20T00:00:00+00:00,1.4,Polygon,"[[12.1848, 55.7244], [12.5026, 55.7157], [12.5...",0035b981-c4ee-a509-988b-8e4e15a36473
2,2024-01-19.txt,2024-02-27T18:11:34.682000,20km_632_48,2024-02-27T18:11:58.158608+00:00,2024-01-20T00:00:00.001000+01:00,min_temp,manual,day,2024-01-21T00:00:00+01:00,0.4,Polygon,"[[8.6706, 57.0231], [9, 57.0235], [9, 57.2032]...",00374820-11cb-5c02-ab40-068c0dfc6e2d
3,2024-01-19.txt,2024-02-28T05:17:59.838000,20km_620_44,2024-02-28T05:32:04.499419+00:00,2024-01-19T23:00:00+00:00,mean_pressure,manual,hour,2024-01-20T00:00:00+00:00,1010.2,Polygon,"[[8.0394, 55.9416], [8.3596, 55.9437], [8.3566...",00412bab-098e-34d4-dc36-f8aadb23bdea
4,2024-01-19.txt,2024-02-27T10:37:24.110000,20km_612_56,2024-02-27T10:39:40.520203+00:00,2024-01-20T00:00:00.001000+01:00,max_temp_w_date,manual,day,2024-01-21T00:00:00+01:00,4.9,Polygon,"[[9.9432, 55.2229], [10.2575, 55.2201], [10.26...",0042242a-b0dc-37f6-383e-1c3eb8d0304b


In [6]:
display(df['parameterId'].unique())

array(['temp_soil_30', 'min_temp', 'mean_pressure', 'max_temp_w_date',
       'max_wind_speed_3sec', 'no_summer_days', 'no_days_acc_precip_10',
       'no_days_acc_precip_1', 'vapour_pressure_deficit_mean',
       'leaf_moisture', 'mean_temp', 'drought_index', 'mean_radiation',
       'mean_wind_speed', 'temp_grass', 'acc_precip', 'mean_cloud_cover',
       'mean_wind_dir', 'acc_heating_degree_days_17',
       'max_wind_speed_10min', 'no_ice_days', 'mean_relative_hum',
       'bright_sunshine', 'temp_soil_10', 'no_tropical_nights',
       'no_days_acc_precip_01', 'pot_evaporation_makkink',
       'max_precip_30m', 'snow_depth'], dtype=object)

In [7]:
# Filter the DataFrame
filtered_df = df[(df['parameterId'] == 'mean_temp') & (df['timeResolution'] == 'day')]

# Display the filtered DataFrame
display(filtered_df)

,file,calculatedAt,cellId,created,from,parameterId,qcStatus,timeResolution,to,value,geometry_type,coordinates,feature_id
18,2024-01-19.txt,2024-02-27T01:32:08.480000,20km_622_44,2024-02-27T01:33:57.948542+00:00,2024-01-20T00:00:00.001000+01:00,mean_temp,manual,day,2024-01-21T00:00:00+01:00,4.5,Polygon,"[[8.0349, 56.1213], [8.3566, 56.1234], [8.3536...",009defbc-19cb-ff82-792d-7a31985267de
203,2024-01-19.txt,2024-02-27T01:32:07.766000,20km_630_46,2024-02-27T01:33:57.691677+00:00,2024-01-20T00:00:00.001000+01:00,mean_temp,manual,day,2024-01-21T00:00:00+01:00,3.0,Polygon,"[[8.3443, 56.8421], [8.6721, 56.8434], [8.6706...",068b3177-4c4f-9219-3113-e51fb4457088
217,2024-01-19.txt,2024-02-27T01:32:06.392000,20km_614_56,2024-02-27T01:33:57.267794+00:00,2024-01-20T00:00:00.001000+01:00,mean_temp,manual,day,2024-01-21T00:00:00+01:00,3.7,Polygon,"[[9.9475, 55.4026], [10.2632, 55.3997], [10.26...",06dc4026-8bda-872e-3618-a97bc3b4230b
230,2024-01-19.txt,2024-02-27T01:32:07.715000,20km_620_70,2024-02-27T01:33:57.676104+00:00,2024-01-20T00:00:00.001000+01:00,mean_temp,manual,day,2024-01-21T00:00:00+01:00,2.1,Polygon,"[[12.1995, 55.9038], [12.5188, 55.8951], [12.5...",0779d4e6-b70b-589d-9f37-7ff0811e02bc
237,2024-01-19.txt,2024-02-27T01:32:07.300000,20km_620_66,2024-02-27T01:33:57.542440+00:00,2024-01-20T00:00:00.001000+01:00,mean_temp,manual,day,2024-01-21T00:00:00+01:00,2.5,Polygon,"[[11.5604, 55.9188], [11.88, 55.9117], [11.893...",07af1e65-27cf-64f9-6813-da278520ef97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405494,2024-01-14.txt,2024-02-27T01:13:01.948000,20km_614_62,2024-02-27T01:13:22.268474+00:00,2024-01-15T00:00:00.001000+01:00,mean_temp,manual,day,2024-01-16T00:00:00+01:00,-1.1,Polygon,"[[10.8945, 55.3916], [11.2101, 55.3863], [11.2...",fb05b697-83bf-3976-97dd-48a735c4ff63
2405526,2024-01-14.txt,2024-02-27T01:13:02.480000,20km_626_56,2024-02-27T01:13:22.449065+00:00,2024-01-15T00:00:00.001000+01:00,mean_temp,manual,day,2024-01-16T00:00:00+01:00,-4.4,Polygon,"[[9.9742, 56.4806], [10.2988, 56.4777], [10.30...",fc0aec52-da27-c0fa-085a-4c1489bb3447
2405563,2024-01-14.txt,2024-02-27T01:13:01.608000,20km_618_56,2024-02-27T01:13:22.168234+00:00,2024-01-15T00:00:00.001000+01:00,mean_temp,manual,day,2024-01-16T00:00:00+01:00,-2.8,Polygon,"[[9.9562, 55.762], [10.2748, 55.7591], [10.280...",fd23c7e2-022a-cb43-33c1-fb39f4f1a7e0
2405591,2024-01-14.txt,2024-02-27T01:13:02.952000,20km_616_52,2024-02-27T01:13:22.587783+00:00,2024-01-15T00:00:00.001000+01:00,mean_temp,manual,day,2024-01-16T00:00:00+01:00,-2.3,Polygon,"[[9.3173, 55.5856], [9.6346, 55.5843], [9.6375...",fdedb999-9ce6-2314-4ba4-41af92b46456


In [18]:
def plot_polygons_from_df(df):
    # Initialize a list to store the polygons and their properties
    polygons = []
    
    # Iterate over the rows in the dataframe to convert coordinates to Polygons
    for index, row in df.iterrows():
        coordinates = row['coordinates']
        
        if coordinates:  # Ensure there are coordinates
            try:
                # Create a Shapely Polygon from the coordinates
                polygon = Polygon(coordinates)
                polygons.append(polygon)
            except ValueError:
                print(f"Invalid polygon at row {index}")
                polygons.append(None)
        else:
            polygons.append(None)

    # Add the polygons as a geometry column to the DataFrame
    df['geometry'] = polygons

    # Drop duplicate polygons to avoid plotting them multiple times
    df_unique = df.drop_duplicates(subset=['geometry'])

    # Extract coordinates in Plotly compatible format (lon, lat for each polygon)
    fig = go.Figure()

    for i, row in df_unique.iterrows():
        polygon_coords = row['coordinates']
        if polygon_coords:
            # Extract the longitude and latitude from the polygon coordinates
            lons, lats = zip(*polygon_coords)

            # Close the polygon by repeating the first point at the end
            lons = list(lons) + [lons[0]]
            lats = list(lats) + [lats[0]]

            # Add the polygon to the Plotly map
            fig.add_trace(go.Scattermapbox(
                fill="toself",
                lon=lons,
                lat=lats,
                mode="lines",
                line=dict(width=1),
                name=row['cellId'],  # Show the parameter ID in the legend
                text=f"Value: {row['value']} Date: {row['from']}",  # Add value as hover info
                hoverinfo="text"
            ))

    # Set up the layout for Plotly Mapbox
    fig.update_layout(
        mapbox_style="carto-positron",  # You can also use "carto-positron", "satellite" etc.
        mapbox_zoom=5,  # Adjust zoom level
        mapbox_center={"lat": 55.0, "lon": 9.5},  # Adjust the center based on your data
        height=600,
        margin={"r":0,"t":0,"l":0,"b":0},
        showlegend=True
    )

    # Show the figure
    fig.show()

In [19]:
# Call the plotting function
plot_polygons_from_df(filtered_df)

/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_4561/2535145423.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

